# FedLib: Heterogeneous Federated Learning using Dynamic Model Pruning and Adaptive Gradient

## Importing supportive libaries
This notebook shows a demo on PyTorch back-end model impelementation.

In the very begining, we import the supporting libraries.

In [ ]:
import torch
import numpy as np
import copy

from fedlib.ve import FEDDFEnv as Simulator
from fedlib.lib import Server, Client
from fedlib.networks import resnet20
from fedlib.lib.sampler import random_sampler
from fedlib.lib.algo import feddp
from fedlib.datasets import partition_data, get_dataloader,get_client_dataloader
from fedlib.utils import get_logger

## Define arguments
Here we define arguments. To show an intuitive example, we show the demo store all the parameters in a dictionary in the following code block.
We also provide APIs for you create your arguments in a `*.yaml` file.

In [ ]:
args = {}
args["n_clients"] = 100
args["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args['sample_fn'] = random_sampler
args['trainer'] = feddp(logger=get_logger())
args['communicator'] = None
args["test_dataset"] = None
args["partition"] = "noniid-labeldir"
args["dataset"] = "cifar10"
args["datadir"] = "./data"
args["beta"] = 0.5
args["batch_size"] = 64
args["global_model"] = resnet20()
args["lr"] = 0.01
args["optimizer"] = "SGD"
args["lr_scheduler"] = "ExponentialLR"

Load test dataset for server, and passing it as an argument

In [ ]:
X_train, y_train, X_test, y_test, net_dataidx_map, traindata_cls_counts = partition_data(
    args["dataset"], args["datadir"], args['partition'], args['n_clients'], beta=args['beta'])
n_classes = len(np.unique(y_train))
train_dl_global, test_dl_global, train_ds_global, test_ds_global = get_dataloader(args["dataset"],
                                                                                    args["datadir"],
                                                                                      args["batch_size"],
                                                                                      32)
args["test_dataset"] = test_dl_global

## Create server and clients objects
Here we use the arguments we defined before, and create server and clients.

In [11]:
server = Server(**args)
clients = {}

data_loaders, test_loaders = get_client_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, net_dataidx_map)


Files already downloaded and verified
key: 71 	, dataid 431 train_ds: 344 test_ds: 87
Files already downloaded and verified
Files already downloaded and verified
key: 72 	, dataid 636 train_ds: 508 test_ds: 128
Files already downloaded and verified
Files already downloaded and verified
key: 73 	, dataid 526 train_ds: 420 test_ds: 106
Files already downloaded and verified
Files already downloaded and verified
key: 74 	, dataid 844 train_ds: 675 test_ds: 169
Files already downloaded and verified
Files already downloaded and verified
key: 75 	, dataid 316 train_ds: 252 test_ds: 64
Files already downloaded and verified
Files already downloaded and verified
key: 76 	, dataid 425 train_ds: 340 test_ds: 85
Files already downloaded and verified
Files already downloaded and verified
key: 77 	, dataid 316 train_ds: 252 test_ds: 64
Files already downloaded and verified
Files already downloaded and verified
key: 78 	, dataid 644 train_ds: 515 test_ds: 129
Files already downloaded and verified
File

In [12]:


for id in range(args["n_clients"]):
    # dataidxs = net_dataidx_map[id]
    args["id"] = id
    # args["trainloader"], _, _, _ = get_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, dataidxs)
    args["trainloader"] = data_loaders[id]
    args["testloader"] =test_loaders[id]
    args["model"] = copy.deepcopy(args["global_model"])
    args['criterion'] = torch.nn.CrossEntropyLoss()
    clients[id] = Client(**args)

    


Client: 0 	Train: 380 	Test: 96
Client: 1 	Train: 490 	Test: 123
Client: 2 	Train: 333 	Test: 84
Client: 3 	Train: 287 	Test: 72
Client: 4 	Train: 206 	Test: 52
Client: 5 	Train: 320 	Test: 80
Client: 6 	Train: 236 	Test: 60
Client: 7 	Train: 232 	Test: 58
Client: 8 	Train: 287 	Test: 72
Client: 9 	Train: 456 	Test: 115
Client: 10 	Train: 147 	Test: 37
Client: 11 	Train: 253 	Test: 64
Client: 12 	Train: 179 	Test: 45
Client: 13 	Train: 627 	Test: 157
Client: 14 	Train: 348 	Test: 88
Client: 15 	Train: 480 	Test: 121
Client: 16 	Train: 415 	Test: 104
Client: 17 	Train: 456 	Test: 115
Client: 18 	Train: 282 	Test: 71
Client: 19 	Train: 401 	Test: 101
Client: 20 	Train: 257 	Test: 65
Client: 21 	Train: 410 	Test: 103
Client: 22 	Train: 502 	Test: 126
Client: 23 	Train: 500 	Test: 126
Client: 24 	Train: 355 	Test: 89
Client: 25 	Train: 268 	Test: 67
Client: 26 	Train: 488 	Test: 123
Client: 27 	Train: 516 	Test: 129
Client: 28 	Train: 472 	Test: 119
Client: 29 	Train: 382 	Test: 96
Client:

## Create simulator

Simulator simulates the virtual federated learning environments, and run server and clients on single device.

In [13]:
simulator = Simulator(server=server, clients=clients, communication_rounds=10,n_clients= 100,sample_rate=.1)

## Run simulator
User API Simulator.run

In [14]:
simulator.run(local_epochs=1,pruning_threshold=1e-3)

INFO:root:*******starting rounds 1 optimization******
INFO:root:Participate Clients: [ 1 60 88  4 69 38 30 33 26 89]
INFO:root:optimize the 1-th clients
INFO:root:Layer: conv1, Sparsity: 0.69%
INFO:root:Layer: layer1.0.conv1, Sparsity: 0.74%
INFO:root:Layer: layer1.0.conv2, Sparsity: 0.56%
INFO:root:Layer: layer1.1.conv1, Sparsity: 0.61%
INFO:root:Layer: layer1.1.conv2, Sparsity: 0.65%
INFO:root:Layer: layer1.2.conv1, Sparsity: 0.95%
INFO:root:Layer: layer1.2.conv2, Sparsity: 0.65%
INFO:root:Layer: layer2.0.conv1, Sparsity: 0.78%
INFO:root:Layer: layer2.0.conv2, Sparsity: 1.05%
INFO:root:Layer: layer2.1.conv1, Sparsity: 0.79%
INFO:root:Layer: layer2.1.conv2, Sparsity: 1.13%
INFO:root:Layer: layer2.2.conv1, Sparsity: 0.79%
INFO:root:Layer: layer2.2.conv2, Sparsity: 0.99%
INFO:root:Layer: layer3.0.conv1, Sparsity: 0.91%
INFO:root:Layer: layer3.0.conv2, Sparsity: 1.41%
INFO:root:Layer: layer3.1.conv1, Sparsity: 1.43%
INFO:root:Layer: layer3.1.conv2, Sparsity: 1.36%
INFO:root:Layer: layer3

Layer: conv1, Sparsity: 0.69%
Layer: layer1.0.conv1, Sparsity: 0.74%
Layer: layer1.0.conv2, Sparsity: 0.56%
Layer: layer1.1.conv1, Sparsity: 0.61%
Layer: layer1.1.conv2, Sparsity: 0.65%
Layer: layer1.2.conv1, Sparsity: 0.95%
Layer: layer1.2.conv2, Sparsity: 0.65%
Layer: layer2.0.conv1, Sparsity: 0.78%
Layer: layer2.0.conv2, Sparsity: 1.05%
Layer: layer2.1.conv1, Sparsity: 0.79%
Layer: layer2.1.conv2, Sparsity: 1.13%
Layer: layer2.2.conv1, Sparsity: 0.79%
Layer: layer2.2.conv2, Sparsity: 0.99%
Layer: layer3.0.conv1, Sparsity: 0.91%
Layer: layer3.0.conv2, Sparsity: 1.41%
Layer: layer3.1.conv1, Sparsity: 1.43%
Layer: layer3.1.conv2, Sparsity: 1.36%
Layer: layer3.2.conv1, Sparsity: 1.33%
Layer: layer3.2.conv2, Sparsity: 1.27%
Model sparsity: 0.95%
Epoch: 0	Loss: 3.851615	Accuracy:0.281250


INFO:root:Layer: conv1, Sparsity: 0.46%
INFO:root:Layer: layer1.0.conv1, Sparsity: 0.87%
INFO:root:Layer: layer1.0.conv2, Sparsity: 0.78%
INFO:root:Layer: layer1.1.conv1, Sparsity: 0.74%
INFO:root:Layer: layer1.1.conv2, Sparsity: 0.74%
INFO:root:Layer: layer1.2.conv1, Sparsity: 0.95%
INFO:root:Layer: layer1.2.conv2, Sparsity: 0.48%
INFO:root:Layer: layer2.0.conv1, Sparsity: 0.89%
INFO:root:Layer: layer2.0.conv2, Sparsity: 1.01%
INFO:root:Layer: layer2.1.conv1, Sparsity: 0.81%
INFO:root:Layer: layer2.1.conv2, Sparsity: 1.12%
INFO:root:Layer: layer2.2.conv1, Sparsity: 0.79%
INFO:root:Layer: layer2.2.conv2, Sparsity: 0.99%
INFO:root:Layer: layer3.0.conv1, Sparsity: 0.90%
INFO:root:Layer: layer3.0.conv2, Sparsity: 1.39%
INFO:root:Layer: layer3.1.conv1, Sparsity: 1.45%
INFO:root:Layer: layer3.1.conv2, Sparsity: 1.38%
INFO:root:Layer: layer3.2.conv1, Sparsity: 1.33%
INFO:root:Layer: layer3.2.conv2, Sparsity: 1.27%
INFO:root:Model sparsity: 0.97%
INFO:root:Epoch: 0	Loss: 3.831669	Accuracy:0.1

Layer: conv1, Sparsity: 0.46%
Layer: layer1.0.conv1, Sparsity: 0.87%
Layer: layer1.0.conv2, Sparsity: 0.78%
Layer: layer1.1.conv1, Sparsity: 0.74%
Layer: layer1.1.conv2, Sparsity: 0.74%
Layer: layer1.2.conv1, Sparsity: 0.95%
Layer: layer1.2.conv2, Sparsity: 0.48%
Layer: layer2.0.conv1, Sparsity: 0.89%
Layer: layer2.0.conv2, Sparsity: 1.01%
Layer: layer2.1.conv1, Sparsity: 0.81%
Layer: layer2.1.conv2, Sparsity: 1.12%
Layer: layer2.2.conv1, Sparsity: 0.79%
Layer: layer2.2.conv2, Sparsity: 0.99%
Layer: layer3.0.conv1, Sparsity: 0.90%
Layer: layer3.0.conv2, Sparsity: 1.39%
Layer: layer3.1.conv1, Sparsity: 1.45%
Layer: layer3.1.conv2, Sparsity: 1.38%
Layer: layer3.2.conv1, Sparsity: 1.33%
Layer: layer3.2.conv2, Sparsity: 1.27%
Model sparsity: 0.97%
Epoch: 0	Loss: 3.831669	Accuracy:0.190625


INFO:root:Layer: conv1, Sparsity: 0.00%
INFO:root:Layer: layer1.0.conv1, Sparsity: 0.82%
INFO:root:Layer: layer1.0.conv2, Sparsity: 0.74%
INFO:root:Layer: layer1.1.conv1, Sparsity: 0.78%
INFO:root:Layer: layer1.1.conv2, Sparsity: 0.74%
INFO:root:Layer: layer1.2.conv1, Sparsity: 0.87%
INFO:root:Layer: layer1.2.conv2, Sparsity: 0.65%
INFO:root:Layer: layer2.0.conv1, Sparsity: 0.85%
INFO:root:Layer: layer2.0.conv2, Sparsity: 0.97%
INFO:root:Layer: layer2.1.conv1, Sparsity: 0.86%
INFO:root:Layer: layer2.1.conv2, Sparsity: 1.07%
INFO:root:Layer: layer2.2.conv1, Sparsity: 0.84%
INFO:root:Layer: layer2.2.conv2, Sparsity: 0.99%
INFO:root:Layer: layer3.0.conv1, Sparsity: 0.88%
INFO:root:Layer: layer3.0.conv2, Sparsity: 1.36%
INFO:root:Layer: layer3.1.conv1, Sparsity: 1.45%
INFO:root:Layer: layer3.1.conv2, Sparsity: 1.32%
INFO:root:Layer: layer3.2.conv1, Sparsity: 1.32%
INFO:root:Layer: layer3.2.conv2, Sparsity: 1.30%
INFO:root:Model sparsity: 0.94%
INFO:root:Epoch: 0	Loss: 2.322350	Accuracy:0.5

...

Layer: conv1, Sparsity: 0.46%
Layer: layer1.0.conv1, Sparsity: 1.22%
Layer: layer1.0.conv2, Sparsity: 1.17%
Layer: layer1.1.conv1, Sparsity: 1.17%
Layer: layer1.1.conv2, Sparsity: 1.35%
Layer: layer1.2.conv1, Sparsity: 1.87%
Layer: layer1.2.conv2, Sparsity: 1.04%
Layer: layer2.0.conv1, Sparsity: 1.19%
Layer: layer2.0.conv2, Sparsity: 1.43%
Layer: layer2.1.conv1, Sparsity: 1.30%
Layer: layer2.1.conv2, Sparsity: 1.64%
Layer: layer2.2.conv1, Sparsity: 1.22%
Layer: layer2.2.conv2, Sparsity: 1.43%
Layer: layer3.0.conv1, Sparsity: 1.26%
Layer: layer3.0.conv2, Sparsity: 1.81%
Layer: layer3.1.conv1, Sparsity: 1.84%
Layer: layer3.1.conv2, Sparsity: 1.73%
Layer: layer3.2.conv1, Sparsity: 1.72%
Layer: layer3.2.conv2, Sparsity: 1.82%
Model sparsity: 1.40%
Epoch: 0	Loss: 1.921493	Accuracy:0.371094


INFO:root:Layer: conv1, Sparsity: 0.46%
INFO:root:Layer: layer1.0.conv1, Sparsity: 1.39%
INFO:root:Layer: layer1.0.conv2, Sparsity: 1.17%
INFO:root:Layer: layer1.1.conv1, Sparsity: 1.17%
INFO:root:Layer: layer1.1.conv2, Sparsity: 1.30%
INFO:root:Layer: layer1.2.conv1, Sparsity: 1.87%
INFO:root:Layer: layer1.2.conv2, Sparsity: 1.04%
INFO:root:Layer: layer2.0.conv1, Sparsity: 1.19%
INFO:root:Layer: layer2.0.conv2, Sparsity: 1.45%
INFO:root:Layer: layer2.1.conv1, Sparsity: 1.28%
INFO:root:Layer: layer2.1.conv2, Sparsity: 1.63%
INFO:root:Layer: layer2.2.conv1, Sparsity: 1.22%
INFO:root:Layer: layer2.2.conv2, Sparsity: 1.42%
INFO:root:Layer: layer3.0.conv1, Sparsity: 1.25%
INFO:root:Layer: layer3.0.conv2, Sparsity: 1.81%
INFO:root:Layer: layer3.1.conv1, Sparsity: 1.83%
INFO:root:Layer: layer3.1.conv2, Sparsity: 1.72%
INFO:root:Layer: layer3.2.conv1, Sparsity: 1.73%
INFO:root:Layer: layer3.2.conv2, Sparsity: 1.81%
INFO:root:Model sparsity: 1.41%
INFO:root:Epoch: 0	Loss: 1.918050	Accuracy:0.3

Layer: conv1, Sparsity: 0.46%
Layer: layer1.0.conv1, Sparsity: 1.39%
Layer: layer1.0.conv2, Sparsity: 1.17%
Layer: layer1.1.conv1, Sparsity: 1.17%
Layer: layer1.1.conv2, Sparsity: 1.30%
Layer: layer1.2.conv1, Sparsity: 1.87%
Layer: layer1.2.conv2, Sparsity: 1.04%
Layer: layer2.0.conv1, Sparsity: 1.19%
Layer: layer2.0.conv2, Sparsity: 1.45%
Layer: layer2.1.conv1, Sparsity: 1.28%
Layer: layer2.1.conv2, Sparsity: 1.63%
Layer: layer2.2.conv1, Sparsity: 1.22%
Layer: layer2.2.conv2, Sparsity: 1.42%
Layer: layer3.0.conv1, Sparsity: 1.25%
Layer: layer3.0.conv2, Sparsity: 1.81%
Layer: layer3.1.conv1, Sparsity: 1.83%
Layer: layer3.1.conv2, Sparsity: 1.72%
Layer: layer3.2.conv1, Sparsity: 1.73%
Layer: layer3.2.conv2, Sparsity: 1.81%
Model sparsity: 1.41%
Epoch: 0	Loss: 1.918050	Accuracy:0.390625


INFO:root:Layer: conv1, Sparsity: 0.46%
INFO:root:Layer: layer1.0.conv1, Sparsity: 1.35%
INFO:root:Layer: layer1.0.conv2, Sparsity: 1.22%
INFO:root:Layer: layer1.1.conv1, Sparsity: 1.09%
INFO:root:Layer: layer1.1.conv2, Sparsity: 1.17%
INFO:root:Layer: layer1.2.conv1, Sparsity: 1.69%
INFO:root:Layer: layer1.2.conv2, Sparsity: 1.04%
INFO:root:Layer: layer2.0.conv1, Sparsity: 1.22%
INFO:root:Layer: layer2.0.conv2, Sparsity: 1.45%
INFO:root:Layer: layer2.1.conv1, Sparsity: 1.30%
INFO:root:Layer: layer2.1.conv2, Sparsity: 1.66%
INFO:root:Layer: layer2.2.conv1, Sparsity: 1.24%
INFO:root:Layer: layer2.2.conv2, Sparsity: 1.44%
INFO:root:Layer: layer3.0.conv1, Sparsity: 1.26%
INFO:root:Layer: layer3.0.conv2, Sparsity: 1.82%
INFO:root:Layer: layer3.1.conv1, Sparsity: 1.84%
INFO:root:Layer: layer3.1.conv2, Sparsity: 1.73%
INFO:root:Layer: layer3.2.conv1, Sparsity: 1.72%
INFO:root:Layer: layer3.2.conv2, Sparsity: 1.82%
INFO:root:Model sparsity: 1.40%
INFO:root:Epoch: 0	Loss: 1.796830	Accuracy:0.3

Layer: conv1, Sparsity: 0.46%
Layer: layer1.0.conv1, Sparsity: 1.35%
Layer: layer1.0.conv2, Sparsity: 1.22%
Layer: layer1.1.conv1, Sparsity: 1.09%
Layer: layer1.1.conv2, Sparsity: 1.17%
Layer: layer1.2.conv1, Sparsity: 1.69%
Layer: layer1.2.conv2, Sparsity: 1.04%
Layer: layer2.0.conv1, Sparsity: 1.22%
Layer: layer2.0.conv2, Sparsity: 1.45%
Layer: layer2.1.conv1, Sparsity: 1.30%
Layer: layer2.1.conv2, Sparsity: 1.66%
Layer: layer2.2.conv1, Sparsity: 1.24%
Layer: layer2.2.conv2, Sparsity: 1.44%
Layer: layer3.0.conv1, Sparsity: 1.26%
Layer: layer3.0.conv2, Sparsity: 1.82%
Layer: layer3.1.conv1, Sparsity: 1.84%
Layer: layer3.1.conv2, Sparsity: 1.73%
Layer: layer3.2.conv1, Sparsity: 1.72%
Layer: layer3.2.conv2, Sparsity: 1.82%
Model sparsity: 1.40%
Epoch: 0	Loss: 1.796830	Accuracy:0.361607
